In [4]:
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
X_train = torch.rand(700, 3, 3)
Y_train = (torch.sum(X_train>0.5, dim=(1,2))>4).float()
X_test = torch.rand(300, 3, 3)
Y_test = (torch.sum(X_test>0.5, dim=(1,2))>4).float()

# data = {}
# data['x_train']=X_train.numpy().tolist()
# data['y_train']=Y_train.numpy().tolist()
# data['x_test']=X_test.numpy().tolist()
# data['y_test']=Y_test.numpy().tolist()

# with open('data.json', 'w') as fp:
#     json.dump(data, fp)

# with open('data.json') as json_file:
#     data = json.load(json_file)

# X_train = torch.FloatTensor(data['x_train'])
# Y_train = torch.FloatTensor(data['y_train'])
# X_test = torch.FloatTensor(data['x_test'])
# Y_test = torch.FloatTensor(data['y_test'])



In [5]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 2, 2)
        self.conv2 = nn.Conv2d(2, 2, 2)
#         self.conv1 = nn.Conv2d(1, 2, 3, padding=1)
#         self.mp1 = nn.MaxPool2d(3, stride=1, padding=1)
#         self.conv2 = nn.Conv2d(2, 3, 3, padding=1)
#         self.mp2 = nn.MaxPool2d(3, stride=1, padding=1)
#         self.mp = nn.MaxPool2d(2, stride=1)
        self.fc = nn.Linear(2, 1)

    def forward(self, x):
        in_size = x.size(0)
#         x = self.mp1(F.relu(self.conv1(x)))
#         x = self.mp2(F.relu(self.conv2(x)))
        
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x

model = ConvNet()

criterion = nn.MSELoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.01)

#
# Print number of parameters
#
for item in list(ConvNet().parameters()):
    print (item)
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

get_n_params(ConvNet())

Parameter containing:
tensor([[[[-0.3138,  0.1925],
          [ 0.3404,  0.2555]]],


        [[[ 0.3131,  0.3707],
          [ 0.2212,  0.3578]]]], requires_grad=True)
Parameter containing:
tensor([0.2025, 0.3994], requires_grad=True)
Parameter containing:
tensor([[[[-0.1215, -0.0530],
          [ 0.1265,  0.1908]],

         [[ 0.3070, -0.1205],
          [-0.0003,  0.2908]]],


        [[[-0.0689, -0.2408],
          [-0.0852, -0.0619]],

         [[-0.2708,  0.1962],
          [ 0.2626, -0.1889]]]], requires_grad=True)
Parameter containing:
tensor([-0.2477, -0.3116], requires_grad=True)
Parameter containing:
tensor([[-0.5839, -0.4292]], requires_grad=True)
Parameter containing:
tensor([-0.3378], requires_grad=True)


31

In [6]:
# 
# Training
# 
results = {} #{iter: [loss, accuracy]}
for epoch in range(1000):  # loop over the dataset multiple times

    running_loss = 0.0
    for i in range(len(Y_train)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = X_train[i], torch.tensor([Y_train[i]])
        inputs = inputs.view(1,1,len(inputs), len(inputs))
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
#         print(outputs.squeeze(0), labels)
        loss = criterion(outputs.squeeze(0), labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    
    if epoch % 10 == 9:
        print ("Epoch: %d, Loss: %.3f"%(epoch+1, running_loss))
    
        correct = 0.
        total = float(len(Y_test))

        with torch.no_grad():
            for i in range(len(Y_test)):
                images, labels = X_test[i], torch.tensor(Y_test[i])
                images = images.view(1,1,len(images), len(images))

                output = model(images)

                predicted = output.data > 0.5

                correct += (int(predicted[0]) == int(labels))
                
                accuracy = 100. * correct / total
        print('Accuracy of the network: %.2f %%' % (accuracy))
        
        results[epoch+1] = [running_loss, accuracy]

Epoch: 10, Loss: 90.429
Accuracy of the network: 82.33 %


/miniconda3/envs/camo/lib/python3.6/site-packages/ipykernel_launcher.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch: 20, Loss: 88.628
Accuracy of the network: 82.33 %
Epoch: 30, Loss: 87.882
Accuracy of the network: 82.67 %
Epoch: 40, Loss: 87.573
Accuracy of the network: 82.67 %
Epoch: 50, Loss: 87.320
Accuracy of the network: 82.67 %
Epoch: 60, Loss: 86.925
Accuracy of the network: 83.00 %
Epoch: 70, Loss: 86.108
Accuracy of the network: 83.33 %
Epoch: 80, Loss: 85.720
Accuracy of the network: 84.33 %
Epoch: 90, Loss: 85.414
Accuracy of the network: 85.00 %
Epoch: 100, Loss: 85.110
Accuracy of the network: 85.33 %
Epoch: 110, Loss: 84.862
Accuracy of the network: 84.67 %
Epoch: 120, Loss: 84.707
Accuracy of the network: 84.67 %
Epoch: 130, Loss: 84.561
Accuracy of the network: 85.00 %
Epoch: 140, Loss: 84.509
Accuracy of the network: 84.67 %
Epoch: 150, Loss: 84.415
Accuracy of the network: 84.67 %
Epoch: 160, Loss: 84.539
Accuracy of the network: 84.67 %
Epoch: 170, Loss: 84.348
Accuracy of the network: 85.33 %
Epoch: 180, Loss: 84.335
Accuracy of the network: 85.33 %
Epoch: 190, Loss: 84.1

In [ ]:
with open('results.json', 'w') as fp:
    json.dump(results, fp)
    
    
    
# correct = 0.
# total = float(len(Y_test))

# outputs = []
# with torch.no_grad():
#     for i in range(len(Y_test)):
#         images, labels = X_test[i], torch.tensor(Y_test[i])
#         images = images.view(1,1,len(images), len(images))
        
#         output = model(images)
#         outputs.append(output)

#         predicted = output.data > 0.5

#         correct += (int(predicted[0]) == int(labels))

# print('Accuracy of the network on the %d test images: %d %%' % (len(Y_test),
#     100 * correct / total))